#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [ ]:
# !pip install pymongo

In [ ]:
import pandas as pd
from pymongo import MongoClient

In [ ]:
client = MongoClient('mongo')
database = client['exercices']
collection = database['kickstarter']

In [ ]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [ ]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Nettoyer les données

In [ ]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv", low_memory = False)
df_ks["launched_year"] = df_ks["launched"]

for i in range(len(df_ks)) :
    df_ks["launched_year"].values[i] = df_ks["launched_year"].values[i][:4]
    
df_ks["launched_year"] = df_ks["launched_year"].astype(float)

for i in range(len(df_ks)) :
    if df_ks["backers"][i] == "successful" :
        df_ks.drop(i, 0, inplace = True)
        
df_ks["backers"] = df_ks["backers"].astype(float)

df_ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,launched_year
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24.0,US,1145.0,1145.000000,2011.0
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28.0,US,1851.0,1851.000000,2012.0
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254.0,US,3796.0,7534.000000,2016.0
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367.0,US,2670.0,16298.000000,2017.0
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1.0,CA,0.70814759,0.738225,2016.0


### Importer les données

In [ ]:
df_json = df_ks.to_json(orient = 'records')
# df_json[0:1000]

In [ ]:
import json
df_json_bis = json.loads(df_json)
# df_json_bis[0:1000]

In [ ]:
collection.insert_many(df_json_bis)

## Question 1  

Récupérer les 5 projets ayant reçu le plus de promesse de dons.

In [ ]:
top_5_pledged = list(collection.find().sort([("usd_pledged_real", -1)])[0:5])

## Question 2

Compter le nombre de projets ayant atteint leur but.

In [ ]:
successful = list(collection.find({"state":"successful"}))
successful_no = len(successful)

## Question 3

Compter le nombre de projets pour chaque catégorie.

In [ ]:
categories = list(collection.aggregate([{"$group" : {"_id" : "$category", "Nombre de projets" : {"$sum" : 1}}}]))

## Question 4

Compter le nombre de projets français ayant été instanciés avant 2016.

In [ ]:
projets_fr_av_2016 = list(collection.find({"launched_year":{"$lt":2016}}, {"country" : "FR"}))
projets_fr_av_2016_no = len(projets_fr_av_2016)

## Question 5

Récupérer les projets américains ayant demandé plus de 200 000 dollars.

In [ ]:
projets_us_plus_200k = list(collection.find({"usd_pledged_real":{"$gt":200000}, "country" : "US"}))

## Question 6 

Compter le nombre de projet ayant "Sport" dans leur nom.

In [ ]:
collection.create_index([("name",  "text")])
sport = list(collection.find({"$text": { "$search": "Sport" }}))